# Declaring Libraries

In [1]:
import pandas                    as pd
import numpy                     as np
import matplotlib.pyplot         as plt
import UnivariateFunctions       as dpm
import time
import pickle

from   sklearn.model_selection   import train_test_split 
from   sklearn.preprocessing     import StandardScaler
from   sklearn.feature_selection import SelectKBest
from   sklearn.feature_selection import chi2
from   sklearn.feature_selection import RFE
from   sklearn.linear_model      import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

# Get Processed Data

In [2]:
# dataset,df,ind,dep,x_train,x_test,y_train,ytest= dpm.getProcessedData()

In [3]:
# # Funciton definition for Feature Selection
# def selectkbest(indep_X, dep_Y, n):
#     test = SelectKBest(score_func=chi2, k=n)
#     fit1 = test.fit(indep_X, dep_Y)
#     selected_features = indep_X.columns[fit1.get_support()]   # <--- Store column names
#     selectk_features = fit1.transform(indep_X)
#     return selectk_features, selected_features

# Funciton definition for Feature Selection
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        selectk_features = fit1.transform(indep_X)
        return selectk_features

# Definition of Scalar Matrix
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

# Definition of Confusion Matric    
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

# Definition for Logistic Regression 
def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      

def svm_gridsearch(X_train,y_train,X_test):
    
    from sklearn.model_selection import GridSearchCV
    from sklearn.svm import SVR
    from sklearn.svm import SVC  # ✅ use SVC, not SVR
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    
    param_grid = {'kernel':['rbf','poly','sigmoid','linear'],
                  'C':[10,100,1000,2000,3000],'gamma':['auto','scale']} 
#     classifier = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3,n_jobs=-1) 

    # Use SVC for classification
    grid = GridSearchCV(
        estimator=SVC(random_state=0),
        param_grid=param_grid,
        cv=2,
        verbose=1,
        n_jobs=1   # ✅ use fewer cores to avoid crashing
    )

    
    
    # fitting the model for grid search 
    grid.fit(X_train, y_train) 
    
    y_pred = grid.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)    
    
#     classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
    return grid.best_estimator_, acc, report, X_test, y_test, cm
#     return  classifier,Accuracy,report,X_test,y_test,cm
    
    
# Definition for SVM Linear Regression  
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

    
    
# Definition of Support Vector Matrix    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
# Definition of Navie Regression
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
# Definition for K-Nearest Neighbor Regression    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
# Definition for Decision Tree Regression
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      

# Defnition for Random Forest Regression
def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def selectk_Classification(acclog,grid,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','GridCV','SVMl_linear','SVMnl_rbf','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]   =acclog[number]       
#         dataframe['GridCV'][idex]     =grid[number]       
        dataframe['SVMl_linear'][idex]=accsvml[number]
        dataframe['SVMnl_rbf'][idex]  =accsvmnl[number]
        dataframe['KNN'][idex]        =accknn[number]
        dataframe['Navie'][idex]      =accnav[number]
        dataframe['Decision'][idex]   =accdes[number]
        dataframe['Random'][idex]     =accrf[number]
    return dataframe


# Reading CSV File as a dataset

In [4]:
dataset=pd.read_csv("HeartFailureDataset v1.csv",index_col=None)

# Data Pre-processing
#### 1) Convert all NaN values to '0' for the columns  ['gluc', 'smoke','alco','active']
#### 2) Convert all NaN values to 'False' or 'True' for the column ['cardio']  as this is to be a Categorical Column
#### 3) Splitting Independent & Dependent dataset
#### 3) Converting Negative (-ve) values to 0 as this is not allowed for classificaiton

In [5]:

df2 = dataset
df2 = pd.get_dummies(dataset, drop_first=True)
df2 = df2.drop('id',axis=1)

# 1) Convert all NaN values to '0' for the columns ['gluc', 'smoke','alco','active']
cols_numeric      = ['gluc', 'smoke','alco','active']
df2[cols_numeric] = np.where(df2[cols_numeric].isna(), 0, 1)

# 2) Column where NaN -> False and others -> True
df2['cardio'] = np.where(df2['cardio'].isna(), False, True)

# Convert age from days to years (whole number)
df2['age'] = (df2['age'] / 365).astype(int)

# Correct: keep original 0/1 values instead of making all True
df2['cardio'] = df2['cardio'].fillna(0).astype(int)
# df2['cardio'] = df2['cardio'].fillna(0).astype(bool)

# 3) Splitting Independent & Dependent dataset
ind  =df2.drop(['cardio'], axis=1)
dep  =df2['cardio']
indep_X= ind 
dep_Y  = dep

# 3) Converting Negative (-ve) values to 0 as this is not allowed for classificaiton
print("ind [Before -ve removal]:-\n",indep_X.min()) # Check if there is any -ve values
indep_X[indep_X < 0] = 0
print("\nind [Afer -ve removal]:-\n",indep_X.min()) # Check if the -ve values are removed



ind [Before -ve removal]:-
 age             29
height          55
weight          30
ap_hi         -140
ap_lo            0
cholesterol      1
gluc             1
smoke            0
alco             0
active           0
gender_Male      0
dtype: int64

ind [Afer -ve removal]:-
 age            29
height         55
weight         30
ap_hi           0
ap_lo           0
cholesterol     1
gluc            1
smoke           0
alco            0
active          0
gender_Male     0
dtype: int64


In [6]:
# Adding 7 -feature parameters  (ie.0-7 = 8 inputs)
# The input-parameters are:
#    'age', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'gender_Male'

kbest=selectkbest(indep_X,dep_Y,8)   
print(kbest)

acclog     =[]
accsvml    =[]
accsvmnl   =[]
accknn     =[]
accnav     =[]
accdes     =[]
accrf      =[]
grid       =[]
classifiers=[]


[[ 50 168  62 ...   1   0   1]
 [ 55 156  85 ...   3   0   1]
 [ 51 165  64 ...   3   0   0]
 ...
 [ 64 168  74 ...   1   0   1]
 [ 60 160 128 ...   2   0   0]
 [ 59 159  64 ...   1   0   1]]


In [7]:
X_train, X_test, y_train, y_test=split_scalar(kbest,dep_Y)   
    
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)
classifiers.append(classifier)

# classifier,Accuracy,report,X_test,y_test,cm=svm_gridsearch(X_train,y_train,X_test)
# grid.append(Accuracy)
# classifiers.append(classifier)


classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
classifiers.append(classifier)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
classifiers.append(classifier)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
classifiers.append(classifier)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
classifiers.append(classifier)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
classifiers.append(classifier)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)
classifiers.append(classifier)
    
result=selectk_Classification(acclog,grid,accsvml,accsvmnl,accknn,accnav,accdes,accrf)


# Displaying the accuracy of every model

In [8]:
from tabulate import tabulate
pd.set_option('display.expand_frame_repr', False)
print(tabulate(result, headers='keys', tablefmt='grid'))


+-----------+------------+----------+---------------+-------------+----------+----------+------------+----------+
|           |   Logistic |   GridCV |   SVMl_linear |   SVMnl_rbf |      KNN |    Navie |   Decision |   Random |
+===========+============+==========+===============+=============+==========+==========+============+==========+
| ChiSquare |   0.722258 |      nan |      0.722911 |    0.726991 | 0.691743 | 0.641808 |   0.642298 | 0.694027 |
+-----------+------------+----------+---------------+-------------+----------+----------+------------+----------+


# Finding the Best Model Automatically

In [9]:
df = pd.DataFrame(result, index=['ChiSquare'])

# ensure all values are numeric
df = df.apply(pd.to_numeric, errors='coerce')

# find best model
best_model = df.loc['ChiSquare'].idxmax()
best_score = df.loc['ChiSquare'].max()
best_index = df.loc['ChiSquare'].values.argmax()

print(f"✅ Best Model: {best_model}")
print(f"📈 Accuracy: {best_score:.6f}")
print(f"🔢 Index Position: {best_index}")

regressor = classifiers[best_index]

✅ Best Model: SVMnl_rbf
📈 Accuracy: 0.726991
🔢 Index Position: 1


# Save and implement the best model found

In [10]:
##-MODEL SAVE: 
import pickle
filename = "CapstoneProject_FeatureSelection.sav"
pickle.dump(regressor,open(filename,'wb'))       ## WRITE BINARY

In [11]:
quan,qual = dpm.getQuanQual(df2)
quan

['age',
 'height',
 'weight',
 'ap_hi',
 'ap_lo',
 'cholesterol',
 'gluc',
 'smoke',
 'alco',
 'active',
 'gender_Male']

In [12]:
##-MODEL LOAD

loaded_model = pickle.load(open(filename,'rb'))  ## READ BINARY


age        =48
height     =169
weight     =82
ap_hi      =150
ap_lo      =100
cholesterol=0
gluc       =0
active     = 0
gender_Male=0


result     =loaded_model.predict([[age,height,ap_hi,ap_lo,cholesterol,gluc,active,gender_Male]])
result

array([1])

In [14]:
if result[0]==1:
    print("Diagonized with Cardiovascular disorder")

Diagonized with Cardiovascular disorder
